In [1]:
from main import get_miner
from log_parsing import from_id_to_template
import joblib
from data_integrate import *
from pattern_miner import *
from pattern_ranker2 import *
import pandas as pd
import json
from utils import *

In [2]:
# Reading from a file
with open("input/root_cause_hipster.json", 'r') as file:
    rc_dict = json.load(file)

In [3]:
log_template_miner = get_miner()

In [4]:
res_dict = joblib.load('output/fault_free_eventgraphs.joblib')

In [5]:
res_dict2 = joblib.load('output/fault_suffering_eventgraphs.joblib')

In [6]:
res_dict2.keys()

dict_keys(['2022-08-22 03:54', '2022-08-22 03:55', '2022-08-22 04:03', '2022-08-22 04:04', '2022-08-22 04:11', '2022-08-22 04:12', '2022-08-22 04:19', '2022-08-22 04:20', '2022-08-22 04:28', '2022-08-22 04:29', '2022-08-22 04:37', '2022-08-22 04:38', '2022-08-22 04:45', '2022-08-22 04:46', '2022-08-22 04:53', '2022-08-22 04:54', '2022-08-22 05:02', '2022-08-22 05:03', '2022-08-22 05:10', '2022-08-22 05:11', '2022-08-22 05:19', '2022-08-22 05:20', '2022-08-22 05:28', '2022-08-22 05:29', '2022-08-22 05:36', '2022-08-22 05:37', '2022-08-22 05:44', '2022-08-22 05:45', '2022-08-22 05:53', '2022-08-22 05:54', '2022-08-22 06:02', '2022-08-22 06:03', '2022-08-22 06:36', '2022-08-22 06:37', '2022-08-22 06:53', '2022-08-22 06:54', '2022-08-22 07:02', '2022-08-22 07:03', '2022-08-22 07:11', '2022-08-22 07:12', '2022-08-22 07:18', '2022-08-22 07:19', '2022-08-22 07:27', '2022-08-22 07:28', '2022-08-22 07:45', '2022-08-22 07:46', '2022-08-22 07:54', '2022-08-22 07:55', '2022-08-23 12:02', '2022-08-

In [7]:
abnormal_dates = res_dict2.keys()

In [8]:
min_score= 0.5

In [9]:
abnormal_dates = res_dict2.keys()
features_labeled = []
for normal_date in res_dict.keys():
    date = normal_date.split(' ')[0]
    fault_free_graphs = res_dict[normal_date]
    fault_free_pattern = get_pattern_support(fault_free_graphs)
    print(date)
    for abnormal_date in [d for d in abnormal_dates if d.split(' ')[0] == date]:
        print(abnormal_date)
        inject_pod = res_dict2[abnormal_date]['inject_pod']
        inject_svc = inject_pod.split("-")[0]
        inject_type = res_dict2[abnormal_date]['inject_type']
        root_cause =  rc_dict[inject_svc][inject_type].split('_')
        alarm_data_path = "input/fault_suffering"
        alarm_metric_list = get_metric_with_time(abnormal_date, alarm_data_path)
        alarm_list = generate_alarm(alarm_metric_list)
        fault_suffering_event_graphs = res_dict2[abnormal_date]['event_graphs']
        fault_suffering_pattern = get_pattern_support(fault_suffering_event_graphs)
        expected_moreoften = pattern_ranker_custom(fault_free_pattern, fault_free_graphs, fault_suffering_pattern, log_template_miner, alarm_list, min_score=min_score)
        actual_moreoften = pattern_ranker_custom(fault_suffering_pattern, fault_suffering_event_graphs, fault_free_pattern, log_template_miner, alarm_list, min_score=min_score)
        metrics_of_affectedpods = get_podmetrics(expected_moreoften, actual_moreoften, abnormal_date, normal_date)
        res = { 
               'normal_time': normal_date,
               'abnormal_time': abnormal_date,
               'expected_moreoften': expected_moreoften,
               'actual_moreoften': actual_moreoften,
               'metrics_of_affectedpods': metrics_of_affectedpods,
               'inject_time': res_dict2[abnormal_date]['inject_time'],
               'inject_pod': inject_pod, 
               'inject_type': inject_type,
               'root_cause': root_cause
            }
        features_labeled.append(res)

2022-08-22
2022-08-22 03:54


/home/paul/miniconda3/envs/ml_pyth36/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


2022-08-22 03:55
2022-08-22 04:03
2022-08-22 04:04
2022-08-22 04:11
2022-08-22 04:12
2022-08-22 04:19
2022-08-22 04:20
2022-08-22 04:28
2022-08-22 04:29
2022-08-22 04:37
2022-08-22 04:38
2022-08-22 04:45
2022-08-22 04:46
2022-08-22 04:53
2022-08-22 04:54
2022-08-22 05:02
2022-08-22 05:03
2022-08-22 05:10
2022-08-22 05:11
2022-08-22 05:19
2022-08-22 05:20
2022-08-22 05:28
2022-08-22 05:29
2022-08-22 05:36
2022-08-22 05:37
2022-08-22 05:44
2022-08-22 05:45
2022-08-22 05:53
2022-08-22 05:54
2022-08-22 06:02
2022-08-22 06:03
2022-08-22 06:36
2022-08-22 06:37
2022-08-22 06:53
2022-08-22 06:54
2022-08-22 07:02
2022-08-22 07:03
2022-08-22 07:11
2022-08-22 07:12
2022-08-22 07:18
2022-08-22 07:19
2022-08-22 07:27
2022-08-22 07:28
2022-08-22 07:45
2022-08-22 07:46
2022-08-22 07:54
2022-08-22 07:55
2022-08-23
2022-08-23 12:02
2022-08-23 12:03
2022-08-23 12:11
2022-08-23 12:12
2022-08-23 12:19
2022-08-23 12:20
2022-08-23 12:28
2022-08-23 12:29
2022-08-23 12:36
2022-08-23 12:37
2022-08-23 12:46
202

In [10]:
# Convert to DataFrame
df = pd.DataFrame(features_labeled)
# locate templates with err or fail in string, treat them special (dont lose on ranking)

In [11]:
df.to_csv(f'output/features_labeledv3.csv', index=False)